In [27]:
from datetime import datetime, timedelta
import pandas as pd
from tqdm import tqdm
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from database.market import Market
from database.adatabase import ADatabase
import numpy as np

In [28]:
market = Market()
speculation_db = ADatabase("bitcoin_speculation")

In [29]:
market.connect()
prices = market.retrieve("pdr_crypto")
market.disconnect()

In [30]:
prices = p.column_date_processing(prices)
prices["year"] = [x.year for x in prices["date"]]
prices["week"] = [x.week for x in prices["date"]]

In [31]:
speculation_db.connect()
simulation = speculation_db.retrieve("pdr_sim")
speculation_db.disconnect()

In [32]:
positions = 1
start_date = datetime(2022,6,17)
end_date = datetime(2022,8,16)
speculation_db.connect()
for value in [True,False]:
    for training_year in [1,4]:
        sim = prices.merge(simulation[simulation["training_year"]==training_year],on=["date","ticker"],how="left").dropna()
        sim["prediction"] = (sim["cat_prediction"] + sim["skl_prediction"] + sim["xgb_prediction"]) / 3
        sim["delta"] = (sim["prediction"] - sim["adjclose"]) / sim["adjclose"]
        for signal in tqdm([0.05,0.1,0.3,0.5]):
            for req in tqdm([0.01,0.03,0.05,0.1]):
                for position in range(positions):
                    date = start_date
                    iterration_sim = sim.copy()
                    if value == False:
                        iterration_sim["delta"] = iterration_sim["delta"] * -1
                    while date < end_date:
                        try:
                            todays_recs = iterration_sim[(iterration_sim["date"]==date)]
                            todays_recs.sort_values("delta",ascending=False,inplace=True)
                            if todays_recs.index.size > 0:
                                offering = todays_recs.iloc[position]
                                if (offering["delta"] > signal):
                                    trade = offering
                                    ticker = trade["ticker"]
                                    buy_price = trade["adjclose"]
                                    end_index = 2
                                    exits = iterration_sim[(iterration_sim["ticker"]==ticker) & (iterration_sim["date"]>date)].iloc[:end_index]
                                    exits["gains"] = (exits["adjclose"] - buy_price) / buy_price
                                    gain_exits = exits[exits["gains"]>=req].sort_values("date")
                                    if gain_exits.index.size < 1:
                                        exit = iterration_sim[(iterration_sim["ticker"]==ticker) & \
                                                              (iterration_sim["date"]>date)].iloc[end_index:].iloc[0]
                                        trade["sell_price"] = exit["adjclose"]
                                    else:
                                        exit = gain_exits.iloc[0]
                                        trade["sell_price"] = buy_price * (1+(req))
                                    delta = (trade["sell_price"] - buy_price) / buy_price
                                    trade["sell_date"] = exit["date"]
                                    date = exit["date"] + timedelta(days=1)
                                    trade["projected_delta"] = offering["delta"]
                                    trade["delta"] = delta
                                    trade["position"] = position
                                    trade["req"] = req
                                    trade["value"] = value
                                    trade["training_year"] = training_year
                                    trade["signal"] = signal
                                    included = ["date","sell_date","ticker","adjclose","sell_price","delta"
                                                ,"projected_delta","position"
                                                ,"req","signal","value","training_year"]
                                    trade_record = pd.DataFrame([trade]).reset_index(drop=True)[included]
                                    speculation_db.store("pdr_trades",trade_record)
                                else:
                                    date = date + timedelta(days=1)
                            else:
                                date = date + timedelta(days=1)
                        except Exception as e:
                            print(str(e))
                            date = date+timedelta(days=1)
speculation_db.disconnect()

 50%|█████████████████████████████████████████████████████████████████████                                                                     | 2/4 [00:00<00:00,  7.48it/s]

single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds



 25%|██████████████████████████████████▌                                                                                                       | 1/4 [00:00<00:01,  2.03it/s]

single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds



 50%|█████████████████████████████████████████████████████████████████████                                                                     | 2/4 [00:00<00:00,  7.84it/s]

...


 50%|█████████████████████████████████████████████████████████████████████                                                                     | 2/4 [00:00<00:00,  8.89it/s]

single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds



 25%|██████████████████████████████████▌                                                                                                       | 1/4 [00:00<00:01,  2.38it/s]

single positional indexer is out-of-bounds



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  7.69it/s]
